<a href="https://colab.research.google.com/github/TheKerbecs/DeepLearning25/blob/main/week_1/CIFAR10-ShallowLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1: CIFAR10 Challenge

**CIFAR10** (http://www.cs.toronto.edu/~kriz/cifar.html) is one of the most famous ML data sets.

## Data
* 32x32 color images
* in 10 classes
* 50k training images
* 10k test images



<img src="https://production-media.paperswithcode.com/datasets/CIFAR-10-0000000431-b71f61c0_U5n3Glr.jpg" width=700>

In [20]:
#get data
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [21]:
#traindata: 50k 32X32 rgb images
X_train.shape

(50000, 32, 32, 3)

In [22]:
#labels
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

## Task: build the best classifier (with feature extration) using the methods you know from ML1+2
* work in small teams (2-4)
* use NumPy pre-processing, feature extraction and hyer-parameter tuning in Scikit-Learn
* no Neural Networks!
* best test F1-Score winns!

In [23]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
import time
from sklearn.decomposition import PCA
import cv2
from skimage.feature import hog
from tqdm import tqdm

In [24]:
def extract_features(images):
    features = []

    features = images.reshape(images.shape[0], -1)
    print(f"Feature shape: {features.shape}")
    return features

In [25]:
X_train_features = extract_features(X_train)
X_test_features = extract_features(X_test)

Feature shape: (50000, 3072)
Feature shape: (10000, 3072)


In [26]:
rf_classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    n_jobs=-1,
    random_state=42
)

rf_classifier.fit(X_train_features, y_train)

# Evaluate the model
print("Evaluating model...")
y_pred = rf_classifier.predict(X_test_features)

# Calculate metrics
f1 = f1_score(y_test, y_pred, average='macro')
print(f"F1 Score: {f1:.4f}")

# Print detailed classification report
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Evaluating model...
Macro F1 Score: 0.4611

Classification Report:
              precision    recall  f1-score   support

    airplane       0.55      0.55      0.55      1000
  automobile       0.51      0.54      0.53      1000
        bird       0.39      0.32      0.35      1000
         cat       0.35      0.25      0.29      1000
        deer       0.39      0.40      0.39      1000
         dog       0.42      0.39      0.40      1000
        frog       0.47      0.59      0.52      1000
       horse       0.50      0.46      0.48      1000
        ship       0.57      0.61      0.59      1000
       truck       0.46      0.54      0.50      1000

    accuracy                           0.47     10000
   macro avg       0.46      0.47      0.46     10000
weighted avg       0.46      0.47      0.46     10000

